In [6]:
import sys
sys.path.append("..")

from main import *
import numpy as np

## step11

In [7]:
# 가변 길이 인수를 받을 수 있도록 조정. 아래는 기존의 Function 클래스
# class Function:
#     def __call__(self, input):          # f = Function() 형태로 함수의 인스턴스를 변수 f에 대입 가능    / input 은 Variable 인스턴스라 가정
#         x = input.data                  # 데이터를 꺼낸다
#         y = self.forward(x)
#         output = Variable(np.array(y))  # step09    / np.ndarray 연산 이후 np.float으로 변경되어서 Variable의 데이터 타입에서 오류 발생 가능하므로 타입 지정
#         output.set_creator(self)        # step07    / 출력 변수에 창조자 설정
#         self.input = input              # step06    / 입력 변수 기억(보관)  / 함수의 앞, 뒤 관계 기억
#         self.output = output            # step07    / f1(x) = output -> f2(output) = y  여기서 "->"이 관계가 output에 들어있다.
#         return output

## step12

In [8]:
# 가변 길이 인수는 튜플로 인식된다.
def f(*x):
    print(x)
    print(type(x))

f(1, 2, 3)

(1, 2, 3)
<class 'tuple'>


## step14

##### Variable 클래스의 역전파 과정에서 ```x.grad += gx``` 는 in-place 연산으로 x.grad의 주소는 유지한 채 그 값만 바꾸는 연산이다.<br>이 때 x.grad의 주소는 앞선 노드의 주소를 그대로 가져왔을 수 있다.(Add클래스의 역전파는 y.grad의 주소를 두 개 리턴한다```return gx, gx```)<br>따라서 이후 y.grad의 계산에 오류가 뜨지 않는 오류가 발생한다.<br>그러므로 가능하면 out-of-place 연산(여기서는 x.grad = x.grad + gx)를 사용하자

## step16

##### 여러 레이어를 쌓고 각 레이어의 입력과 출력 엣지가 복잡하게 연결될 경우 기존의 역전파 알고리즘은 제대로 작동하지 않는다.<br>그래서 각 변수와 함수에 세대(레이어)를 입력하고 세대 순으로 역전파를 진행하도록 코드를 수정한다.<br>seen_set은 funcs리스트에 같은 함수가 반복해서 들어가지 않도록 하기 위함이다.

In [9]:
x = Variable(np.array(2))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)
print(y.generation)
print(x.generation)
print(a.generation)
print(y.creator.inputs[0].generation)       # square(a).generation

32
64
3
0
1
2


## step17

##### Function 인스턴스는 두 개의 Variable인스턴스(입력과 출력)를 참조한다. <br>그리고 output인스턴스는 creator로 Function 인스턴스를 참조하므로 여기서 순환 참조 관계가 생성된다.<br>이를 제거하기위해 output 참조 관계를 약한 참조로 만든다. 그러면 이후 순환 참조 관계가 없으므로 메모리에 남아 방해되는 것이 없다.

## step18 : 메모리 절약 모드

In [10]:
# 역전파 비활성화 기능
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)
    print(x.grad)

None


## step19 : 변수 사용성 개선
> ndarray 인스턴스 변수처럼 print, len, .shape등을 사용 가능하도록 Variable클래스에 메서드 추가